In [1]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [1]:
from ortools.sat.python import cp_model

# Crear el modelo
model = cp_model.CpModel()

# Definir variables
start_times = {}
end_times = {}
intervals = []
makespan = model.NewIntVar(0, 100, "makespan")

load c:\Users\Monica\Documents\Kit Consulting\Sergar\desarrollo\notebooks\venv\lib\site-packages\ortools\.libs\zlib1.dll...
load c:\Users\Monica\Documents\Kit Consulting\Sergar\desarrollo\notebooks\venv\lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\Users\Monica\Documents\Kit Consulting\Sergar\desarrollo\notebooks\venv\lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\Users\Monica\Documents\Kit Consulting\Sergar\desarrollo\notebooks\venv\lib\site-packages\ortools\.libs\re2.dll...
load c:\Users\Monica\Documents\Kit Consulting\Sergar\desarrollo\notebooks\venv\lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\Users\Monica\Documents\Kit Consulting\Sergar\desarrollo\notebooks\venv\lib\site-packages\ortools\.libs\highs.dll...
load c:\Users\Monica\Documents\Kit Consulting\Sergar\desarrollo\notebooks\venv\lib\site-packages\ortools\.libs\ortools.dll...


In [2]:
def planificar_produccion(pedidos):
    model = cp_model.CpModel()

    start_times = {}
    end_times = {}
    task_intervals = []
    all_tasks = []

    makespan = model.NewIntVar(0, 100, "makespan")

    for pedido, data in pedidos.items():
        prev_end = None
        for i, (proceso, duracion) in enumerate(data["procesos"]):
            start = model.NewIntVar(0, data["fecha_entrega"], f"start_{pedido}_{i}")
            end = model.NewIntVar(0, data["fecha_entrega"], f"end_{pedido}_{i}")
            interval = model.NewIntervalVar(start, int(duracion), end, f"interval_{pedido}_{i}")

            if prev_end is not None:
                model.Add(start >= prev_end)

            prev_end = end
            start_times[(pedido, i)] = start
            end_times[(pedido, i)] = end
            task_intervals.append(interval)
            all_tasks.append((pedido, i, start))

    model.AddMaxEquality(makespan, [end_times[key] for key in end_times])
    model.Minimize(makespan)

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        plan = []
        for (pedido, i), start in start_times.items():
            plan.append((solver.Value(start), pedido, i, pedidos[pedido]["nombre"], pedidos[pedido]["cantidad"], pedidos[pedido]["procesos"][i][0]))

        plan.sort()
        print("Orden de producción:")
        for start_time, pedido, i, nombre_pedido, cantidad, nombre_proceso in plan:
            print(f"Pedido {pedido} ({nombre_pedido}, {cantidad} unidades), Proceso {i} ({nombre_proceso}): Comienza en {start_time}")

        print(f"Tiempo total de producción: {solver.Value(makespan)}")
    else:
        print("No se encontró solución")

In [3]:
# Ejemplo de pedidos
pedidos = {
    1: {"nombre": "Placa aluminio de 60x75x100", "cantidad": 110, "fecha_entrega": 24, "procesos": [("Dibujo", 3.648), ("Impresión", 3.648), ("Taladro", 1.824)]},
    2: {"nombre": "Caja de madera 40x50x60", "cantidad": 50, "fecha_entrega": 30, "procesos": [("Dibujo", 2), ("Corte", 4), ("Barniz", 1)]},
}

planificar_produccion(pedidos)

Orden de producción:
Pedido 1 (Placa aluminio de 60x75x100, 110 unidades), Proceso 0 (Dibujo): Comienza en 0
Pedido 2 (Caja de madera 40x50x60, 50 unidades), Proceso 0 (Dibujo): Comienza en 0
Pedido 2 (Caja de madera 40x50x60, 50 unidades), Proceso 1 (Corte): Comienza en 2
Pedido 1 (Placa aluminio de 60x75x100, 110 unidades), Proceso 1 (Impresión): Comienza en 3
Pedido 1 (Placa aluminio de 60x75x100, 110 unidades), Proceso 2 (Taladro): Comienza en 6
Pedido 2 (Caja de madera 40x50x60, 50 unidades), Proceso 2 (Barniz): Comienza en 6
Tiempo total de producción: 7
